In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('Restaurant_Reviews.tsv', sep='\t')
df.head()

,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [3]:
import re
import nltk
from nltk.stem import PorterStemmer

In [4]:
ps= PorterStemmer()

In [5]:
def get_processed(msg):
    msg= re.sub('[^a-zA-Z]',' ', msg)
    msg=msg.lower()
    list_A= msg.split()
    list_B= [ps.stem(word) for word in list_A if word not in nltk.corpus.stopwords.words('english')]
    msg=' '.join(list_B)
    return msg

In [6]:
df['Review'].apply(get_processed)

0                                         wow love place
1                                             crust good
2                                     tasti textur nasti
3      stop late may bank holiday rick steve recommen...
4                                select menu great price
                             ...                        
995                        think food flavor textur lack
996                               appetit instantli gone
997                         overal impress would go back
998    whole experi underwhelm think go ninja sushi n...
999    wast enough life pour salt wound draw time too...
Name: Review, Length: 1000, dtype: object

In [7]:
#Build TF-IDF model
from sklearn.feature_extraction.text import TfidfVectorizer

In [9]:
tf_idf= TfidfVectorizer()

In [10]:
tf_idf_table= tf_idf.fit_transform(df['Review'].apply(get_processed))

In [11]:
tf_idf_table

<1000x1565 sparse matrix of type '<class 'numpy.float64'>'
	with 5372 stored elements in Compressed Sparse Row format>

In [12]:
X= tf_idf_table.toarray()

In [13]:
y= df['Liked']

In [14]:
#Build ML Model
from sklearn.ensemble import RandomForestClassifier

In [15]:
model= RandomForestClassifier(random_state=56)

In [16]:
model.fit(X,y)

RandomForestClassifier(random_state=56)

In [17]:
model.score(X,y)

0.997

In [18]:
def get_sentiment(review):
    review= get_processed(review)
    a1= tf_idf.transform([review])
    a1= a1.toarray()
    return model.predict(a1)[0]

In [19]:
get_sentiment('Food was very nice but not near to my place')

1

In [20]:
get_sentiment('Service was not good')

1

In [21]:
get_sentiment('did not like at all')

0

In [22]:
get_sentiment('I did not like the place')

1